In [11]:
import xarray as xr

load in a raster (image) dataset. disregard lack of spatial data warning. using rasterio as a quick and dirty way to get what is an image loaded into in as xarray data array

In [12]:
ts = xr.open_rasterio('frame_4000.png', parse_coordinates=None, chunks=None, cache=None, lock=None)

/opt/conda/lib/python3.6/site-packages/rasterio/__init__.py:193: UserWarning: Dataset has no geotransform set.  Default transform will be applied (Affine.identity())
  s.start()


convert to dataset

In [13]:
ts = ts.to_dataset(name = 'image')

write to zarr.   
this creates a local folder called `camhd_frames` that contains the zarr data. interestingly, it requires that I specify `camhd` as the group, so somehow it must know that it is working on the server, however i don't see these data being written to the `camhd_frames` container on azure.

so instead of creating a local folder, `to_zarr` also lets you write to a MutableMapping object.
http://xarray.pydata.org/en/stable/generated/xarray.Dataset.to_zarr.html
https://github.com/zarr-developers/zarr/blob/f8df70ddcd714d544a913e3420279adf66b4994d/zarr/creation.py

this is probable where elements from the azrfs map come in, though we need to figure out what exactly should be passed in here. 
https://github.com/friedrichknuth/rte-camhd/blob/master/examples/azure_file_mapping.ipynb

In [15]:
ts_za = ts.to_zarr(store='frame_4000.zarr',mode='w', synchronizer=None, group='camhd', encoding=None)

trying to write the folder to blob storage doesn't work. it seems to need a single file or possibly virtual file. 

In [20]:
from azure.storage.blob import BlockBlobService
azure_storage_account_name = ''
azure_storage_account_key = ''
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)

blob_service.create_blob_from_path('camhd_frames','frame_4000.zarr', 'frame_4000.zarr')

IsADirectoryError: [Errno 21] Is a directory: 'frame_4000.zarr'

whatever the correct function is, it will likely need to be packed under the hood of what is started in azrfs, would be my guess, along with MutableMapping functionality
https://github.com/friedrichknuth/rte-camhd/tree/master/examples/azrfs